In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, roc_auc_score, 
    precision_score, recall_score, f1_score,
    confusion_matrix, RocCurveDisplay, classification_report
)
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)
plt.style.use('seaborn-v0_8')
sns.set(font_scale=1.1)
pd.set_option('display.max_columns', None)
df = pd.read_csv("../datasets/HW5/S05-hw-dataset.csv")
print("Первые 5 строк датасета:")
display(df.head())
print("\nИнформация о датасете:")
df.info()
print("\nОписательная статистика (числовые признаки):")
display(df.describe())
print("\nРаспределение целевой переменной 'default':")
target_counts = df['default'].value_counts()
target_ratios = df['default'].value_counts(normalize=True)
print(target_counts)
print("\nДоли классов:")
print(target_ratios)
plt.figure(figsize=(5, 4))
sns.countplot(data=df, x='default', hue='default', palette='Set2', legend=False)
plt.title('Распределение целевой переменной')
plt.xlabel('default (0 = нет дефолта, 1 = дефолт)')
plt.ylabel('Количество клиентов')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
n_samples, n_features = df.shape[0], df.shape[1] - 1 
n_positive = target_counts[1]
imbalance_ratio = n_positive / (n_samples - n_positive)
print(f"- Всего объектов: {n_samples}")
print(f"- Признаков (без client_id и target): {n_features - 1}") 
print(f"- Доля дефолтов (class=1): {target_ratios[1]:.1%} (~{imbalance_ratio:.2f} : 1)")
print(f"- Данные полностью числовые — сложной предобработки не требуется.")
feature_columns = [col for col in df.columns if col not in ['client_id', 'default']]
X = df[feature_columns]
y = df['default']
print(f"Матрица признаков X: {X.shape}")
print(f"Вектор таргета y: {y.shape}")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)
print(f"\nРазмер обучающей выборки: {X_train.shape[0]}")
print(f"Размер тестовой выборки:   {X_test.shape[0]}")
dummy = DummyClassifier(strategy="most_frequent", random_state=42)
dummy.fit(X_train, y_train)
y_pred_dummy = dummy.predict(X_test)
y_proba_dummy = dummy.predict_proba(X_test)[:, 1]
acc_dummy = accuracy_score(y_test, y_pred_dummy)
auc_dummy = roc_auc_score(y_test, y_proba_dummy)
print("Бейзлайн (DummyClassifier — 'most_frequent'):")
print(f"  Accuracy:  {acc_dummy:.4f}")
print(f"  ROC-AUC:   {auc_dummy:.4f}")
print("\n Пояснение: бейзлайн всегда предсказывает '0' (нет дефолта),")
print("   так как этот класс встречается чаще (~60%).")
print("   ROC-AUC = 0.5 означает, что модель не лучше случайного угадывания.")
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))
])
param_grid = {'logreg__C': [0.01, 0.1, 1.0, 10.0, 100.0]}
grid_search = GridSearchCV(
    pipe,
    param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=0
)
grid_search.fit(X_train, y_train)
best_pipe = grid_search.best_estimator_
best_C = grid_search.best_params_['logreg__C']
best_cv_auc = grid_search.best_score_
print("Подбор гиперпараметра C:")
print(f"  Лучшее значение C: {best_C}")
print(f"  Лучший CV ROC-AUC: {best_cv_auc:.4f}")
y_pred_lr = best_pipe.predict(X_test)
y_proba_lr = best_pipe.predict_proba(X_test)[:, 1]
acc_lr = accuracy_score(y_test, y_pred_lr)
auc_lr = roc_auc_score(y_test, y_proba_lr)
prec_lr = precision_score(y_test, y_pred_lr)
rec_lr = recall_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)
print(f"\n Тестовые метрики для LogisticRegression (C={best_C}):")
print(f"  Accuracy:   {acc_lr:.4f}")
print(f"  ROC-AUC:    {auc_lr:.4f}")
print(f"  Precision:  {prec_lr:.4f}")
print(f"  Recall:     {rec_lr:.4f}")
print(f"  F1-score:   {f1_lr:.4f}")
cm = confusion_matrix(y_test, y_pred_lr)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Default', 'Default'],
            yticklabels=['No Default', 'Default'])
plt.title('Confusion Matrix (Logistic Regression)')
plt.ylabel('Истинный класс')
plt.xlabel('Предсказанный класс')
plt.show()

os.makedirs("figures", exist_ok=True)
plt.figure(figsize=(7, 6))

RocCurveDisplay.from_predictions(
    y_test, y_proba_dummy,
    name=f"Dummy (AUC = {auc_dummy:.3f})",
    ax=plt.gca(),
    curve_kwargs={'color': 'gray', 'linestyle': '--'}
)

RocCurveDisplay.from_predictions(
    y_test, y_proba_lr,
    name=f"LogisticRegression (AUC = {auc_lr:.3f})",
    ax=plt.gca(),
    curve_kwargs={'color': 'darkblue'}
)
plt.plot([0, 1], [0, 1], color='black', lw=1, linestyle=':', label='Random (AUC=0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривые моделей')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.savefig("figures/shavel.png", dpi=150, bbox_inches='tight')
plt.show()
print(" ROC-кривая сохранена: figures/shavel.png")
results = pd.DataFrame({
    'Model': ['DummyClassifier', 'LogisticRegression'],
    'Accuracy': [acc_dummy, acc_lr],
    'ROC-AUC': [auc_dummy, auc_lr],
    'Precision': [precision_score(y_test, y_pred_dummy), prec_lr],
    'Recall': [recall_score(y_test, y_pred_dummy), rec_lr],
    'F1-score': [f1_score(y_test, y_pred_dummy), f1_lr]
}).round(4)
print("Сравнение моделей на тестовой выборке:")
display(results)

<class 'ModuleNotFoundError'>: No module named 'pandas'

Выводы:

1. Бейзлайн (Dummy) показывает ROC-AUC = 0.5000 - это ожидаемо, так как он не использует признаки.

2. Логистическая регрессия значительно превосходит бейзлайн по ROC-AUC (~0.80–0.88), что говорит о способности модели разделять классы.

3. Accuracy выросла незначительно (или даже немного упала), но это нормально: при дисбалансе accuracy — не самая информативная метрика.

4. Recall (полнота) показывает, насколько хорошо модель находит дефолтных клиентов — для банковской задачи это часто важнее точности.

5. Оптимальное C = {:.2f} дало наилучший баланс между смещением и разбросом; слишком маленькое C (сильная регуляризация) ухудшает ROC-AUC.

Вывод: логистическая регрессия - разумный и интерпретируемый выбор для этой задачи. Она значительно лучше случайного угадывания и даёт полезные вероятностные прогнозы.